# <center> Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 5.  Соревнование Kaggle "Identify Me If You Can"

На этой неделе мы вспомним про концепцию стохастического градиентного спуска и опробуем классификатор Scikit-learn SGDClassifier, который работает намного быстрее на больших выборках, чем алгоритмы, которые мы тестировали на 4 неделе. Также мы познакомимся с данными [соревнования](https://inclass.kaggle.com/c/identify-me-if-you-can-yandex-mipt/) Kaggle по идентификации пользователей и сделаем в нем первые посылки. По итогам этой недели дополнительные баллы получат те, кто попадет в топ-30 публичного лидерборда соревнования.

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. Sklearn.linear_model. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
**Также рекомендуется вернуться и просмотреть [задание](https://www.coursera.org/learn/supervised-learning/programming/t2Idc/linieinaia-rieghriessiia-i-stokhastichieskii-ghradiientnyi-spusk) "Линейная регрессия и стохастический градиентный спуск" 1 недели 2 курса специализации.**

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, f1_score
import pickle
%pylab inline

Populating the interactive namespace from numpy and matplotlib


**Считаем данные [соревнования](https://inclass.kaggle.com/c/identify-me-if-you-can-yandex-mipt/data) в DataFrame train_df и test_df (обучающая и тестовая выборки).**

In [2]:
train_df = pd.read_csv('kaggle_data/train_sessions.csv', index_col='session_id')
test_df = pd.read_csv('kaggle_data/test_sessions.csv', index_col='session_id')

In [3]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,user_id
session_id,,,,,,,,,,,,,,,,,,,,,
1,8,2014-01-04 08:44:50,11.0,2014-01-04 08:44:50,82.0,2014-01-04 08:45:19,68.0,2014-01-04 08:45:25,8.0,2014-01-04 08:45:25,...,2014-01-04 08:45:51,8403.0,2014-01-04 08:45:51,932.0,2014-01-04 08:45:53,3260.0,2014-01-04 08:45:53,8.0,2014-01-04 08:45:53,1845
2,111,2014-03-18 10:33:20,78.0,2014-03-18 10:33:31,151.0,2014-03-18 10:33:31,111.0,2014-03-18 10:33:31,1401.0,2014-03-18 10:33:31,...,2014-03-18 10:33:32,1375.0,2014-03-18 10:33:32,38.0,2014-03-18 10:33:32,1401.0,2014-03-18 10:33:32,97.0,2014-03-18 10:33:34,3322
3,11,2014-12-02 13:13:41,3187.0,2014-12-02 13:13:41,132.0,2014-12-02 13:13:42,496.0,2014-12-02 13:13:42,1969.0,2014-12-02 13:13:45,...,2014-12-02 13:13:45,3187.0,2014-12-02 13:13:45,82.0,2014-12-02 13:13:46,3191.0,2014-12-02 13:13:46,3184.0,2014-12-02 13:13:47,2003
4,668,2014-02-14 15:16:45,1965.0,2014-02-14 15:17:13,598.0,2014-02-14 15:20:47,1965.0,2014-02-14 15:21:13,284.0,2014-02-14 15:21:14,...,2014-02-14 15:21:14,38.0,2014-02-14 15:21:14,4451.0,2014-02-14 15:21:14,4537.0,2014-02-14 15:21:15,11.0,2014-02-14 15:21:15,1373
5,1943,2014-03-17 15:19:40,1943.0,2014-03-17 15:20:10,1943.0,2014-03-17 15:21:40,1943.0,2014-03-17 15:22:10,1943.0,2014-03-17 15:22:39,...,2014-03-17 15:22:39,1952.0,2014-03-17 15:22:41,1943.0,2014-03-17 15:22:41,1943.0,2014-03-17 15:22:42,1943.0,2014-03-17 15:22:43,1737


**Объединим обучающую и тестовую выборки – это понадобится, чтоб вместе потом привести их к разреженному формату.**

In [5]:
train_test_df = pd.concat([train_df, test_df])

В обучающей выборке видим следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - user_id – ID пользователя
    
Сессии пользователей выделены таким образом, что они не могут быть длинее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд, либо когда сессия заняла по времени более 30 минут. 

**Посмотрим на статистику признаков.**

Пропуски возникают там, где сессии короткие (менее 10 сайтов). Скажем, если человек 1 января 2015 года посетил *vk.com* в 20:01, потом *yandex.ru* в 20:29, затем *google.com* в 20:33, то первая его сессия будет состоять только из двух сайтов (site1 – ID сайта *vk.com*, time1 – 2015-01-01 20:01:00, site2 – ID сайта  *yandex.ru*, time2 – 2015-01-01 20:29:00, остальные признаки – NaN), а начиная с *google.com* пойдет новая сессия, потому что уже прошло более 30 минут с момента посещения *vk.com*.

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95319 entries, 1 to 95319
Data columns (total 21 columns):
site1      95319 non-null int64
time1      95319 non-null object
site2      93722 non-null float64
time2      93722 non-null object
site3      92339 non-null float64
time3      92339 non-null object
site4      91085 non-null float64
time4      91085 non-null object
site5      89868 non-null float64
time5      89868 non-null object
site6      88776 non-null float64
time6      88776 non-null object
site7      87755 non-null float64
time7      87755 non-null object
site8      86738 non-null float64
time8      86738 non-null object
site9      85754 non-null float64
time9      85754 non-null object
site10     84810 non-null float64
time10     84810 non-null object
user_id    95319 non-null int64
dtypes: float64(9), int64(2), object(10)
memory usage: 16.0+ MB


In [7]:
test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,1917,2014-12-04 20:36:21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20225,2014-03-04 14:08:55,27.0,2014-03-04 14:08:59,9.0,2014-03-04 14:08:59,32.0,2014-03-04 14:09:50,20225.0,2014-03-04 14:10:43,20258.0,2014-03-04 14:10:44,20225.0,2014-03-04 14:10:45,20225.0,2014-03-04 14:11:07,20276.0,2014-03-04 14:11:08,20225.0,2014-03-04 14:11:09
3,71,2014-03-31 09:20:38,63.0,2014-03-31 09:20:42,64.0,2014-03-31 09:20:43,63.0,2014-03-31 09:20:43,71.0,2014-03-31 09:20:43,22.0,2014-03-31 09:20:45,64.0,2014-03-31 09:20:46,71.0,2014-03-31 09:20:47,71.0,2014-03-31 09:20:48,70.0,2014-03-31 09:20:51
4,7338,2014-02-14 13:39:36,7338.0,2014-02-14 14:07:53,7338.0,2014-02-14 14:08:36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,19622,2014-03-31 09:46:04,32.0,2014-03-31 09:46:12,340.0,2014-03-31 09:46:13,19634.0,2014-03-31 09:46:14,1721.0,2014-03-31 09:46:14,327.0,2014-03-31 09:46:15,38.0,2014-03-31 09:46:19,1721.0,2014-03-31 09:46:19,340.0,2014-03-31 09:46:20,19634.0,2014-03-31 09:46:20


In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41177 entries, 1 to 41177
Data columns (total 20 columns):
site1     41177 non-null int64
time1     41177 non-null object
site2     39529 non-null float64
time2     39529 non-null object
site3     38158 non-null float64
time3     38158 non-null object
site4     37030 non-null float64
time4     37030 non-null object
site5     36049 non-null float64
time5     36049 non-null object
site6     35083 non-null float64
time6     35083 non-null object
site7     34284 non-null float64
time7     34284 non-null object
site8     33434 non-null float64
time8     33434 non-null object
site9     32633 non-null float64
time9     32633 non-null object
site10    31907 non-null float64
time10    31907 non-null object
dtypes: float64(9), int64(1), object(10)
memory usage: 6.6+ MB


**В обучающей выборке – 550 пользователей.**

In [9]:
train_df['user_id'].nunique()

550

In [10]:
train_df['user_id'].value_counts().head()

3165    3351
537     1927
3324    1534
1826    1528
1845    1452
Name: user_id, dtype: int64

**Пока для прогноза ID пользователя будем использовать только индексы посещенных сайтов. Индексы нумеровались с 1, так что заменим пропуски на нули.**

In [11]:
train_test_df_sites = train_test_df[['site1', 'site2', 'site3', 
                                     'site4','site5', 
                                     'site6','site7', 'site8', 
                                     'site9', 'site10']].fillna(0).astype('int')

In [26]:
print len(train_test_df_sites)
print(len(train_df), len(test_df))

136496
(95319, 41177)


**Создайте разреженные матрицы *X_train_sparse* и *X_test_sparse* аналогично тому, как мы это делали ранее. Используйте объединенную матрицу train_test_df_sites – потом разделите обратно на обучающую и тестовую части.**

Обратите внимание на то, что в  сессиях меньше 10 сайтов  у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных (сколько раз попался сайт с индексом $i$). Поэтому первый столбец разреженной матрицы надо будет удалить.

**Выделите в отдельный вектор *y* ответы на обучающей выборке.**

In [14]:
train_test_df_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,8,11,82,68,8,8393,8403,932,3260,8
2,111,78,151,111,1401,151,1375,38,1401,97
3,11,3187,132,496,1969,504,3187,82,3191,3184
4,668,1965,598,1965,284,668,38,4451,4537,11
5,1943,1943,1943,1943,1943,1952,1952,1943,1943,1943


In [8]:
def sparsematrix(X):
    indptr = [0]
    indices = []
    data = []
    vocabulary = {}
    for d in X:
        for term in d:
            if term != 0:
                index = vocabulary.setdefault(term, len(vocabulary))
                indices.append(index)
                data.append(1)
        indptr.append(len(indices))
    return csr_matrix((data, indices, indptr), dtype=int)

In [43]:
X_train_test_sparse = sparsematrix(train_test_df_sites.as_matrix())
X_train_sparse = X_train_test_sparse[:len(train_df)]
X_test_sparse = X_train_test_sparse[len(train_df):]
y = train_df["user_id"]

In [46]:
X_train_sparse.shape

(95319, 24052)

In [44]:
def write_answer_to_file(answer, file_address):
    with open(file_address, 'w') as out_f:
        out_f.write(str(answer))

**Запишите в файл *answer5_1.txt* размерности матриц *X_train_sparse* и *X_test_sparse* – 4 числа на одной строке через пробел: число строк и столбцов матрицы *X_train_sparse*, затем число строк и столбцов матрицы *X_test_sparse*. Полученный файл будет ответом на 1 вопрос теста.**

In [48]:
write_answer_to_file("{} {} {} {}".format(X_train_sparse.shape[0], X_train_sparse.shape[1], \
                                          X_test_sparse.shape[0], X_test_sparse.shape[1]),
                      'answer5_1.txt')

**Сохраним в pickle-файлы объекты *X_train_sparse*, *X_test_sparse* и *y* (последний – в файл *kaggle_data/train_target.pkl*).**

In [50]:
import pickle

with open('kaggle_data/X_train_sparse.pkl', 'wb') as X_train_sparse_pkl:
    pickle.dump(X_train_sparse, X_train_sparse_pkl)
with open('kaggle_data/X_test_sparse.pkl', 'wb') as X_test_sparse_pkl:
    pickle.dump(X_test_sparse, X_test_sparse_pkl)
with open('kaggle_data/train_target.pkl', 'wb') as train_target_pkl:
    pickle.dump(y, train_target_pkl)

**Разобьем обучающую выборку на 2 части в пропорции 7/3.**

In [51]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_sparse, y, test_size=0.3, 
                                                     random_state=7, stratify=y)

**Создайте объекты sklearn.linear_model.SGDClassifier с логистической функцией потерь и с hinge loss (логистическая регрессия и линейный SVM соответственно) и параметром random_state=7. Остальные параметры оставьте по умолчанию, разве что n_jobs=-1 никогда не помешает. Обучите  модели на выборке (X_train, y_train).**

In [52]:
sgd_logit = SGDClassifier(loss='log', random_state=7, n_jobs=-1)

In [53]:
%%time
sgd_logit.fit(X_train, y_train)

CPU times: user 39.3 s, sys: 288 ms, total: 39.6 s
Wall time: 5.34 s


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=-1,
       penalty='l2', power_t=0.5, random_state=7, shuffle=True, verbose=0,
       warm_start=False)

In [54]:
sgd_svm = SGDClassifier(loss='hinge', random_state=7, n_jobs=-1)

In [55]:
%%time
sgd_svm.fit(X_train, y_train)

CPU times: user 27.8 s, sys: 308 ms, total: 28.1 s
Wall time: 3.89 s


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=-1,
       penalty='l2', power_t=0.5, random_state=7, shuffle=True, verbose=0,
       warm_start=False)

**Сделаем прогнозы с помощью обеих моделей на отложенной выборке (X_valid, y_valid).**

In [56]:
logit_valid_pred = sgd_logit.predict(X_valid)
svm_valid_pred = sgd_svm.predict(X_valid)

In [26]:
print(accuracy_score(y_valid, logit_valid_pred))
print(accuracy_score(y_valid, svm_valid_pred))

NameError: name 'logit_valid_pred' is not defined

**Запишите в файл *answer5_2.txt* через пробел доли правильных ответов логистической регресии и линейного SVM, обученных с помощью стохастического градиентного спуска, на отложенной выборке. Округлите до 3 знаков после разделителя. Полученный файл будет ответом на 2 вопрос теста.**

In [60]:
write_answer_to_file("{} {}".format(round(accuracy_score(y_valid, logit_valid_pred), 3), round((accuracy_score(y_valid, svm_valid_pred)), 3)),
                     'answer5_2.txt')

**Сделайте прогноз для тестовой выборки с помощью sgd_logit.**

In [61]:
logit_test_pred = sgd_logit.predict(X_test_sparse)

**Запишите ответы в файл и сделайте посылку на Kaggle. Далее будет короткое Peer-Review задание, в котором надо проверить посылки друг друга на Kaggle. Поэтому дайте своей команде (из одного человека) на Kaggle говорящее название – по шаблону "[YDF & MIPT] Coursera Username", чтоб можно было идентифицировать Вашу посылку на [лидерборде](https://inclass.kaggle.com/c/identify-me-if-you-can-yandex-mipt/leaderboard).**

**Результат, который мы только что получили, соответствует бейзлайну "SGDCLassifer" на лидерборде, задача на эту неделю – как минимум его побить, дополнительные баллы будут для тех, кто попадет в топ-10 и топ-30 по итогам этой недели.**

In [29]:
def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [100]:
write_to_submission_file(logit_test_pred, "kaggle_data/first_submission.csv")

## Пути улучшения
На этой неделе дается много времени на соревнование. Не забывайте вносить хорошие идеи, к которым Вы пришли по ходу соревнования, в финальный проект (.pdf или .ipynb).
Что можно попробовать:
 - Использовать ранее построенные признаки для улучшения модели (проверить их можно на меньшей выборке по 150 пользователям – это быстрее)
 - Настроить параметры моделей (например, коэффициенты регуляризации)
 - Если позволяют мощности (или хватает терпения), можно попробовать смешивание (блендинг) ответов бустинга и линейной модели. [Вот](http://mlwave.com/kaggle-ensembling-guide/) один из самых известных тьюториалов по смешиванию ответов алгоритмов
 - Обратите внимание, что в соревновании также даны исходные данные о посещенных 550 пользователями веб-страницах (550 csv-файлов в *train.zip*). По этим данным можно сформировать свою обучающую выборку. 

На 6 неделе мы пройдем тьюториал по Vowpal Wabbit и попробуем его в деле – в соревновании.

In [2]:
from glob import glob
import re

In [3]:
def sitefreq(sites, site_freq={}, site_index={}):
    if not len(site_index):
        if len(site_freq):
            site_id = max(site_freq.items(), key=lambda t: t[1][0])[1][0] + 1
        else:
            site_id = 1
    for site in sites:
        if len(site_index):
            site_id = site_index[site]
                
        if site not in site_freq:
            site_freq[site] = [site_id, 1]
            site_id += 1
        else:
            site_freq[site][1] += 1
            
    return site_freq

In [4]:
def get_site_index(csv_file):
    site_index = pd.read_csv(csv_file, header=None, names=['siteid', 'site'], index_col=1)
    site_dic = site_index.to_dict("dic")
    return site_dic["siteid"]

In [75]:
[[k, v] for k, v in site_freq.items()]

[['archives.lesechos.fr', [7288, 5]],
 ['scholar.google.fr', [4827, 25]],
 ['9zouxfza1h.s.ad6media.fr', [4462, 5]],
 ['blogs.uclouvain.be', [20976, 1]],
 ['www.btb.termiumplus.gc.ca', [14747, 3]],
 ['afecaa4b0c028e55502bc297410239d4d.profile.mad50.cloudfront.net',
  [11419, 1]],
 ['www.decoration-stickers.com', [13196, 5]],
 ['notretribuencolombie.blogspot.fr', [20188, 1]],
 ['www.terra-symbiosis.org', [6491, 1]],
 ['www.bwin.fr', [2105, 36]],
 ['www.edubourse.com', [15629, 1]],
 ['fantom-xp.net', [8548, 1]],
 ['cdn2.free-power-point-templates.com', [9545, 4]],
 ['images.onthesnow.com', [16842, 5]],
 ['montblanc.rambler.ru', [7872, 149]],
 ['thesasreference.wordpress.com', [20452, 1]],
 ['www.bakebooks.com', [8798, 2]],
 ['matchdaymail.ercrugby.com', [14239, 1]],
 ['i1-js-14-3-01-10013-268436885-i.init.cedexis-radar.net', [802, 1]],
 ['webdesignerwall.com', [15105, 10]],
 ['www.map-france.com', [2803, 3]],
 ['sjs.sinajs.cn', [13951, 4]],
 ['assets2.ecologie-shop.com', [6074, 3]],
 ['ww

In [23]:
len([[k, v] for k, v in sorted(site_freq.items(), key=lambda t: t[1][1], reverse=False) if v[1] < 100])

20696

In [75]:
a = np.array([1, 2, 4])
np.where(a>2)[0][0]

2

In [61]:
def prepare_train_set_with_fe(csv_files_mask, feature_names, site_freq_path="", site_index_path="", dataframe_csv="",
                                    session_length=10, window_size=10, session_time = 30, num_users_for_prediction = 1, sort_in_session=False):
    
    global_window_size = window_size
    train_data = np.array([np.zeros(len(feature_names))])
    site_freq = {}
    site_index = {}
    freq_only = False
    session_num = 0
    temp_session_length = session_length
    user_count = 1
    
    if site_freq_path != "":
        pkl_file = open(site_freq_path, 'rb')
        site_freq = pickle.load(pkl_file)
    if site_index_path != "":
        site_index = get_site_index(site_index_path)
    
    if len(site_freq):
        regex=re.compile(".*(facebook).*")
        facebook_ix = [site_freq[i][0] for i in [m.group(0) for l in site_freq.keys() for m in [regex.search(l)] if m]]
        regex=re.compile(".*(youtube).*")
        youtube_ix = [site_freq[i][0] for i in [m.group(0) for l in site_freq.keys() for m in [regex.search(l)] if m]]

        top30_ix = [v[0] for k, v in sorted(site_freq.items(), key=lambda t: t[1][1], reverse=True)[:30]]
        num_less_100 = len([[k, v] for k, v in sorted(site_freq.items(), key=lambda t: t[1][1], reverse=False) if v[1] < 100])
        bot30_ix = [v[0] for k, v in sorted(site_freq.items(), key=lambda t: t[1][1], reverse=False)[:num_less_100]]
    else:
        freq_only = True
        print "Building site freq."
    
    data = None
    if dataframe_csv != "":
        data = pd.read_csv(dataframe_csv, parse_dates=[0], infer_datetime_format=True)
        files = [0]
    else:
        files = glob(csv_files_mask)
        
    for userfile in files:
        if dataframe_csv == "":
            window_size = global_window_size
            user_id = re.search('user(\d+)\.csv', userfile).group(1)
            data = pd.read_csv(userfile, \
                               parse_dates=[0], infer_datetime_format=True)
            #if user_id == "3165":
                #data = data.sample(int(len(data)/2))
            if not len(data):
                continue
            data.sort_values("timestamp", inplace=True)
            data.reset_index(drop=True, inplace=True)
            
            #user_sites_num[user_id] = round(len(data)/10., 0)
            #continue
            
            #factor = int(np.max(user_sites_num.values()) / user_sites_num[user_id])
            #if factor <= 10:
                #sites_window_size[user_id] = int(10/factor)
            #else:
                #sites_window_size[user_id] = 1
                #sites_repetitions[user_id] = int(factor/10) + 1
            #continue
            
            #window_size = sites_window_size[user_id]
            if user_id in sites_repetitions:
                rep = sites_repetitions[user_id]
            else:
                rep = 1
            print "#", user_count, "user_id:", user_id, "window_size:", window_size, "reps:", rep
            user_count += 1
        session_hash = {}    
        
        if site_freq_path == "" and site_index_path == "":
            site_freq = sitefreq(data.site, site_freq)
        elif site_index_path != "":
            site_freq = sitefreq(data.site, site_freq, site_index)
            
        if not freq_only:
        
            session = []
            timestamps = []
            next_session = []
            next_t_session = []
            next_s_session = []
            next_timestamps = []

            for i in range (0, len(data), window_size):  
                session += list(data[i:i+session_length].site.apply(lambda x: site_freq[x][0]))
                timestamps += list(data[i:i+session_length].timestamp)

                if window_size < session_length and sort_in_session:
                    ses_ts_zip = sorted(set(zip(session, timestamps)), key = lambda t: t[1])
                    session, timestamps = zip(*ses_ts_zip)
                    session = list(session)
                    timestamps = list(timestamps)

                while ((len(session) >= session_length) or (not len(data[i+window_size:]) and len(session))):
                    time_diff = [(timestamps[n+1] - timestamps[n]).total_seconds() for n in range(0, len(session)-1)]
                    session_timespan = (max(timestamps) - min(timestamps)).total_seconds()
                    next_session = []
                    next_timestamps = []
                    #print "1", session, session_timespan
                    
                    if session_num in [40152, 40153] and dataframe_csv != "":
                        session_length = 5
                    elif session_num > 40153 and dataframe_csv != "":
                        session_length = temp_session_length
                    
                    while session_timespan > session_time*60 or len(session) > session_length:
                        next_session.insert(0, session.pop())
                        next_timestamps.insert(0, timestamps.pop())
                        time_diff = [(timestamps[n+1] - timestamps[n]).total_seconds() for n in range(0, len(session)-1)]
                        session_timespan = (max(timestamps) - min(timestamps)).total_seconds()
                        #print "2", next_session, session_timespan
                    session = tuple(session)

                    if session not in session_hash or dataframe_csv != "":                      
                        session_hash[session] = 1
                        session = list(session)                       
                                                
                        if dataframe_csv != "" and sort_in_session: #need to sort sites in sessions by timestamp in test data
                            ses_ts_zip = sorted(set(zip(session, timestamps)), key = lambda t: t[1])
                            session, timestamps = zip(*ses_ts_zip)
                            session = list(session)
                            timestamps = list(timestamps)
                            time_diff = [(timestamps[n+1] - timestamps[n]).total_seconds() for n in range(0, len(session)-1)]
                            session_timespan = (max(timestamps) - min(timestamps)).total_seconds()
                        
                        num_unique_sites = len(set(session))
                        start_hour = min(timestamps).hour
                        day_of_week = min(timestamps).weekday()

                        #print "3", session, session_timespan
                        #return None
                        
                        #сайт, на котором пользователь находился дольше всего в сессии
                        if len(session) == 1:
                            site_longest_time = session[0]
                        else:
                            site_longest_time = session[time_diff.index(max(time_diff))]
                        
                        #доля facebook в сессии по времени
                        facebook_in_session = np.where(np.in1d(session, facebook_ix) == True)[0]
                        facebook_times = [time_diff[t] for t in facebook_in_session if t < len(time_diff)]
                        fb_portion = sum(facebook_times)/session_timespan if len(facebook_times) and session_timespan else 0
                        #print(facebook_in_session)

                        #доля youtube в сессии по времени
                        youtube_in_session = np.where(np.in1d(session, youtube_ix) == True)[0]
                        youtube_times = [time_diff[t] for t in youtube_in_session if t < len(time_diff)]
                        youtube_portion = sum(youtube_times)/session_timespan if len(youtube_times) and session_timespan else 0

                        #доля топ30 сайтов в сессии по времени
                        top30_in_session = np.where(np.in1d(session, top30_ix) == True)[0]
                        top30_times = [time_diff[t] for t in top30_in_session if t < len(time_diff)]
                        top30_portion = sum(top30_times)/session_timespan if len(top30_times) and session_timespan else 0
                        
                        #доля бот30 сайтов в сессии по времени
                        bot30_in_session = np.where(np.in1d(session, bot30_ix) == True)[0]
                        bot30_times = [time_diff[t] for t in bot30_in_session if t < len(time_diff)]
                        bot30_portion = sum(bot30_times)/session_timespan if len(bot30_times) and session_timespan else 0

                        #время суток начала сессии: 5-12 утро, 12-17 - день, 18-23 вечер, 0-5 ночь
                        if start_hour in range(5,12):
                            daytime = 0
                        elif start_hour in range(12, 18):
                            daytime = 1
                        elif start_hour > 18:
                            daytime = 2
                        elif start_hour < 5:
                            daytime = 3
                            
                        if dataframe_csv != "":
                            session_length = temp_session_length
                            
                            
                        session_predictions = []
                        session_prediction = 0
                        for site in session:
                            if site in site_dic and len(site_dic[site]) == 1:
                                session_prediction = list(site_dic[site])[0]
                                break
                            elif site in site_dic and len(site_dic[site]) < num_users_for_prediction:
                                predictions = sorted([[int(user), float(user_dic[user][site]) / sum(user_dic[user].values())] \
                                                     for user in site_dic[site] if site in user_dic[user]], key=lambda t: t[1], reverse=True)
                                if len(predictions):
                                    session_predictions.append(predictions[0])
                                    
                        if len(session_predictions):
                            session_prediction = sorted(session_prediction, key=lambda t: t[1], reverse=True)[0][0]
                            

                        session.extend([0] * (session_length - len(session)) + \
                                       timestamps + [0] * (session_length - len(timestamps)) + \
                                       time_diff + \
                                       [0]*(session_length - len(time_diff) - 1) + \
                                       [session_timespan, num_unique_sites, site_longest_time, start_hour, day_of_week, daytime, fb_portion,\
                                youtube_portion, top30_portion, bot30_portion, session_prediction])
                        if dataframe_csv == "":
                            session.extend([user_id])
                        
                        if user_id in sites_repetitions:
                            rep = sites_repetitions[user_id]
                            rep_session = np.tile(np.array([session]),(rep,1))
                            train_data = np.concatenate((train_data, rep_session))
                        else:
                            train_data = np.concatenate((train_data, np.array([session])))
                        session_num += 1
                        #print train_data
                        #return None

                    session = next_session
                    timestamps = next_timestamps
            
            if len(next_session):
                print("ERROR! next_session left!")
                print(session)
                return None

    if freq_only:
        with open('kaggle_data/site_freq.pkl', 'wb') as site_freq_pkl:
            pickle.dump(site_freq, site_freq_pkl)
        return site_freq
    
    train_data = np.delete(train_data, 0, 0)
    train_data = pd.DataFrame(train_data, columns=feature_names)
    return train_data

In [62]:
user_sites_num = {}

In [63]:
sites_window_size = {}
sites_repetitions = {}

In [64]:
sites_repetitions

{}

In [45]:
sorted(sites_window_size.items(), key = lambda t: t[1])

[]

In [69]:
del user_sites_num["3165"]

In [103]:
np.std(user_sites_num.values())

211.79158181789384

In [39]:
sorted(user_sites_num.items(), key = lambda t: t[1])

[('2986', 0.0),
 ('1013', 0.0),
 ('1793', 0.0),
 ('2081', 0.0),
 ('1835', 0.0),
 ('937', 0.0),
 ('177', 0.0),
 ('2690', 0.0),
 ('945', 0.0),
 ('1964', 0.0),
 ('1104', 0.0),
 ('673', 0.0),
 ('2394', 0.0),
 ('1738', 0.0),
 ('3219', 0.0),
 ('1329', 0.0),
 ('1700', 1.0),
 ('2383', 1.0),
 ('1339', 1.0),
 ('1750', 1.0),
 ('3113', 1.0),
 ('1307', 1.0),
 ('378', 1.0),
 ('823', 1.0),
 ('797', 1.0),
 ('2270', 1.0),
 ('1963', 1.0),
 ('774', 1.0),
 ('313', 1.0),
 ('387', 1.0),
 ('2031', 1.0),
 ('280', 1.0),
 ('1411', 1.0),
 ('2784', 1.0),
 ('2179', 1.0),
 ('2676', 1.0),
 ('728', 1.0),
 ('1808', 1.0),
 ('2003', 1.0),
 ('1364', 1.0),
 ('2631', 2.0),
 ('239', 2.0),
 ('1505', 2.0),
 ('324', 2.0),
 ('328', 2.0),
 ('1265', 2.0),
 ('692', 2.0),
 ('121', 2.0),
 ('786', 2.0),
 ('2633', 2.0),
 ('1814', 2.0),
 ('55', 2.0),
 ('1413', 2.0),
 ('538', 2.0),
 ('3118', 3.0),
 ('412', 3.0),
 ('2938', 3.0),
 ('265', 3.0),
 ('1566', 3.0),
 ('97', 5.0),
 ('1053', 5.0),
 ('2342', 5.0),
 ('2539', 5.0),
 ('17', 5.0),
 ('

In [46]:
feature_names = ['site' + str(i) for i in range(1,11)] + ['time' + str(i) for i in range(1,11)] + \
                ['time_diff' + str(j) for j in range(1,10)] + \
                ['session_timespan', '#unique_sites', 'site_longest_time', 'start_hour', 'day_of_week', 'daytime', 'fb_portion',\
                 'youtube_portion', 'top30_portion', 'bot30_portion', 'prediction', 'target']

In [55]:
%%time

train_data_toy  = prepare_train_set_with_fe('kaggle_data/train_toy/*',
                                   feature_names=feature_names, 
                                            site_freq_path="kaggle_data/site_freq.pkl", session_length=10, window_size=9)

CPU times: user 716 ms, sys: 0 ns, total: 716 ms
Wall time: 716 ms


In [95]:
train_data_toy.sample(10)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,...,site_longest_time,start_hour,day_of_week,daytime,fb_portion,youtube_portion,top30_portion,bot30_portion,prediction,target
8,22,77,22,22,22,22,10527,22,146,22,...,22,12,2,1,0.0769231,0,0.923077,0.0769231,2135,2135
13,38,32,245,251,77,77,55,22,240,22,...,240,12,2,1,0,0,0.428571,0,0,2135
10,237,55,32,8,237,237,237,255,38,240,...,255,12,2,1,0,0.722222,0.277778,0,0,2135
26,22,521,22,523,521,521,526,524,526,523,...,22,13,2,1,0,0,0.8,0,0,2135
11,38,255,243,237,8,8,251,243,251,239,...,251,12,2,1,0,0.117647,0.882353,0,0,2135
14,146,22,1493,10528,3190,3190,39,40,1969,255,...,1969,12,2,1,0,0,0.342857,0.0714286,2135,2135
27,69,523,524,524,521,521,523,0,0,0,...,524,13,2,1,0,0,0.104478,0,0,2135
17,22,90,69,22,8,8,8,32,32,8,...,22,13,2,1,0,0,1,0,0,2135
25,1601,11,7,11,38,38,88,82,11,22,...,11,13,2,1,0.987342,0,0.987342,0.0126582,0,2135
0,16,22,16,77,22,0,0,0,0,0,...,22,11,2,0,0,0,0.993243,0,0,2135


In [56]:
len(train_data_toy) #user_id 2135 count 245

28

In [21]:
len(train_data_toy)

25

In [19]:
len(train_data_toy) #user_id 3165 count 32564 w10

3138

In [17]:
len(train_data_toy) #user_id 3165 count 32564 w5

5946

In [62]:
train_data_toy.to_csv("kaggle_data/toy_train_w5_old.csv", index=False)

In [142]:
train_data_toy.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,...,#unique_sites,site_longest_time,start_hour,day_of_week,daytime,fb_portion,youtube_portion,top30_portion,bot30_portion,target
0,3537,3537,3537,5759,37,3537,16,17,47,53,...,7,5759,14,4,1,0,0,0,0,280
1,37,56,47,3537,53,47,53,17,22,6,...,8,22,14,4,1,0,0,0.939394,0,280
2,22,22,22,90,22,22,87,22,14326,14326,...,4,87,14,4,1,0,0,0.5,0,280
3,14326,87,14334,14326,0,0,0,0,0,0,...,3,87,14,4,1,0,0,0,0,280
4,540,16,56,47,47,53,17,1027,347,16,...,8,17,15,4,1,0,0,0,0,280


In [5]:
train_data = pd.read_csv("kaggle_data/full_train.csv")

In [6]:
%%time
user_dic = {}
site_dic = {}

pkl_file = open("kaggle_data/site_freq.pkl", 'rb')
site_freq = pickle.load(pkl_file)
top_sites = [v[1] for k, v in sorted(site_freq.items(), key=lambda t: t[1][1], reverse=True)[:0]]

for i, v in train_data.iterrows():
    if v.target not in user_dic:
        user_dic[v.target] = {}
    for site in ['site' + str(i) for i in range(1,11)]:
        if v[site] != 0 and v[site] not in top_sites:
            if v[site] in user_dic[v.target]:
                user_dic[v.target][v[site]] +=1
            else:
                user_dic[v.target][v[site]] = 1
        
        if v[site] in site_dic:
            site_dic[v[site]].add(v.target)
        else:
            site_dic[v[site]] = set([v.target])

CPU times: user 1min 2s, sys: 252 ms, total: 1min 2s
Wall time: 1min 2s


In [199]:
accuracy_score(train_data["target"].astype(int), train_data["prediction"].astype(int))

0.24361915844247087

In [82]:
len(site_freq)

21532

In [169]:
%%time
train_data  = prepare_train_set_with_fe('kaggle_data/train/*',
                                   feature_names=feature_names, 
                                            site_freq_path="kaggle_data/site_freq.pkl", session_length=10)

CPU times: user 30min 31s, sys: 364 ms, total: 30min 31s
Wall time: 30min 31s


**Let's build training sets with window_size 5 and 7.**

In [69]:
%%time
train_data  = prepare_train_set_with_fe('kaggle_data/new_train/*',
                                   feature_names=feature_names, 
                                            site_freq_path="kaggle_data/site_freq.pkl", session_length=10, window_size=7, sort_in_session=True)

#train_data_w5  = prepare_train_set_with_fe('kaggle_data/train/*',
                                   #feature_names=feature_names, 
                                            #site_freq_path="kaggle_data/site_freq.pkl", session_length=10, window_size=5, sort_in_session=True)
#train_data_w7  = prepare_train_set_with_fe('kaggle_data/train/*',
                                   #feature_names=feature_names, 
                                            #site_freq_path="kaggle_data/site_freq.pkl", session_length=10, window_size=7, sort_in_session=True)
        
#train_data_w3  = prepare_train_set_with_fe('kaggle_data/train/*',
                                   #feature_names=feature_names, 
                                            #site_freq_path="kaggle_data/site_freq.pkl", session_length=10, window_size=3)

# 1 user_id: 280 window_size: 7 reps: 1
# 2 user_id: 2517 window_size: 7 reps: 1
# 3 user_id: 2116 window_size: 7 reps: 1
# 4 user_id: 74 window_size: 7 reps: 1
# 5 user_id: 1969 window_size: 7 reps: 1
# 6 user_id: 2947 window_size: 7 reps: 1
# 7 user_id: 1737 window_size: 7 reps: 1
# 8 user_id: 189 window_size: 7 reps: 1
# 9 user_id: 3385 window_size: 7 reps: 1
# 10 user_id: 1580 window_size: 7 reps: 1
# 11 user_id: 174 window_size: 7 reps: 1
# 12 user_id: 2661 window_size: 7 reps: 1
# 13 user_id: 1200 window_size: 7 reps: 1
# 14 user_id: 940 window_size: 7 reps: 1
# 15 user_id: 2708 window_size: 7 reps: 1
# 16 user_id: 2639 window_size: 7 reps: 1
# 17 user_id: 2773 window_size: 7 reps: 1
# 18 user_id: 2631 window_size: 7 reps: 1
# 19 user_id: 3285 window_size: 7 reps: 1
# 20 user_id: 1173 window_size: 7 reps: 1
# 21 user_id: 3119 window_size: 7 reps: 1
# 22 user_id: 1645 window_size: 7 reps: 1
# 23 user_id: 1410 window_size: 7 reps: 1
# 24 user_id: 1745 window_size: 7 reps: 1
# 25 us

In [20]:
np.append([[1, 2, 3]], [[7, 8, 9]] * 2, axis=0)

array([[1, 2, 3],
       [7, 8, 9],
       [7, 8, 9]])

In [41]:
from collections import Counter
number_of_users = Counter(train_data.target).values()

In [42]:
max(number_of_users), min(number_of_users), np.mean(number_of_users), np.std(number_of_users)

(5946, 31, 324.9818181818182, 463.27819213274552)

In [70]:
#train_data.to_csv("kaggle_data/full_train.csv", index=False)
#train_data.to_csv("kaggle_data/full_train_wvar.csv", index=False)
train_data.to_csv("kaggle_data/full_train_w7_min2.csv", index=False)
#train_data_w5.to_csv("kaggle_data/full_train_w5_nosort.csv", index=False)
#train_data_w3.to_csv("kaggle_data/full_train_w3_old.csv", index=False)
#train_data_w7.to_csv("kaggle_data/full_train_w7.csv", index=False)
#train_data_w8.to_csv("kaggle_data/full_train_w8.csv", index=False)

In [14]:
train_data = pd.read_csv("kaggle_data/full_train.csv")

In [187]:
test_data = pd.read_csv("kaggle_data/full_test.csv")

In [7]:
len(test_data)

41177

In [16]:
%%time
test_feature_names = ['site' + str(i) for i in range(1,11)] + ['time' + str(i) for i in range(1,11)] + \
                ['time_diff' + str(j) for j in range(1,10)] + \
                ['session_timespan', '#unique_sites', 'site_longest_time', 'start_hour', 'day_of_week', 'daytime', 'fb_portion',\
                 'youtube_portion', 'top30_portion', 'bot30_portion', 'prediction']
test_data  = prepare_train_set_with_fe('*',
                                   feature_names=test_feature_names, dataframe_csv='kaggle_data/full_test_temp.csv',
                                            site_freq_path="kaggle_data/site_freq.pkl",
                                            site_index_path="kaggle_data/site_indexes.txt", session_length=10)

CPU times: user 8min 23s, sys: 14.1 s, total: 8min 37s
Wall time: 8min 37s


In [90]:
site_indexes = pd.read_csv("kaggle_data/site_indexes.txt", header=None, names=["index", "site"])

In [95]:
import re

In [150]:
top_domains = {}
sec_domains = {}
full_sites = {}
t = 1
s = 1
for r, row in site_indexes.iterrows():
    full_sites[row["index"]] = row["site"]
    if re.search("\d+$", row["site"]):
        continue
    top_domain = re.search("(\w+)$", row["site"]).group(1)
    sec_domain = re.search("(\w+\.\w+)$", row["site"]).group(1)
    if top_domain not in top_domains:
        top_domains[top_domain] = t
        t+=1
    if sec_domain not in sec_domains:
        sec_domains[sec_domain] = s
        s+=1

In [182]:
train_data_w8 = pd.read_csv("kaggle_data/full_train_w8.csv")

In [144]:

top_domains[re.search("(\w+)$", full_sites[train_data_w8.iloc[0,0]]).group(1)]

54

In [153]:
train_data_w8[["t_site"+str(c) for c in range(1,11)]] = 0

KeyError: "['t_site1' 't_site2' 't_site3' 't_site4' 't_site5' 't_site6' 't_site7'\n 't_site8' 't_site9' 't_site10'] not in index"

In [188]:
t_sites = []
s_sites = []
for r, row in test_data[["site"+str(c) for c in range(1,11)]].iterrows():
    t_site = []
    s_site = []
    for site in ["site"+str(c) for c in range(1,11)]:
        if int(row[site]) == 0:
            continue
        if re.search("([0-9]+)$", full_sites[row[site]]):
            t_site.append(0)
            s_site.append(0)
            continue
        t_site.append(top_domains[re.search("(\w+)$", full_sites[row[site]]).group(1)])
        s_site.append(sec_domains[re.search("(\w+\.\w+)$", full_sites[row[site]]).group(1)])
    t_sites.append(t_site)
    s_sites.append(s_site)

In [189]:
test_data = pd.concat([test_data, pd.DataFrame(t_sites, columns=["t_site"+str(c) for c in range(1,11)]).fillna(0), \
                           pd.DataFrame(s_sites, columns=["s_site"+str(c) for c in range(1,11)]).fillna(0)], axis=1)

In [190]:
test_data.to_csv("kaggle_data/full_test_tssite.csv", index=False)

In [168]:
t_sites = []
s_sites = []
for r, row in train_data_w8[["site"+str(c) for c in range(1,11)]].iterrows():
    t_site = []
    s_site = []
    for site in ["site"+str(c) for c in range(1,11)]:
        if int(row[site]) == 0:
            continue
        if re.search("([0-9]+)$", full_sites[row[site]]):
            t_site.append(0)
            s_site.append(0)
            continue
        t_site.append(top_domains[re.search("(\w+)$", full_sites[row[site]]).group(1)])
        s_site.append(sec_domains[re.search("(\w+\.\w+)$", full_sites[row[site]]).group(1)])
    t_sites.append(t_site)
    s_sites.append(s_site)

#test_data.to_csv('kaggle_data/full_test.csv', index=False)

In [183]:
train_data_w8 = pd.concat([train_data_w8, pd.DataFrame(t_sites, columns=["t_site"+str(c) for c in range(1,11)]).fillna(0), \
                           pd.DataFrame(s_sites, columns=["s_site"+str(c) for c in range(1,11)]).fillna(0)], axis=1)

In [184]:
pd.DataFrame(s_sites, columns=["s_site"+str(c) for c in range(1,11)]).fillna(0)

,s_site1,s_site2,s_site3,s_site4,s_site5,s_site6,s_site7,s_site8,s_site9,s_site10
0,1563,1563.0,1563.0,2518.0,6.0,6.0,1563.0,6.0,6.0,6.0
1,6,6.0,6.0,6.0,1563.0,6.0,6.0,6.0,6.0,6.0
2,16,6.0,6.0,6.0,16.0,16.0,16.0,16.0,57.0,16.0
3,16,16.0,16.0,16.0,6274.0,6274.0,16.0,6274.0,6274.0,6274.0
4,6274,6274.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,311,6.0,6.0,6.0,6.0,6.0,6.0,470.0,203.0,6.0
6,203,203.0,57.0,16.0,15.0,8.0,8.0,8.0,8.0,8.0
7,8,8.0,15.0,8.0,8.0,15.0,8.0,8.0,8.0,8.0
8,15,15.0,8.0,8.0,8.0,8.0,8.0,8.0,15.0,8.0
9,8,15.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0


In [186]:
train_data_w8.to_csv("kaggle_data/full_train_w8_tssite.csv", index=False)

In [ ]:
#Now in main function
%%time

for data in [test_data, train_data]:
    data["prediction"] = 0
    for r, v in data.iterrows():
        session_prediction = []
        for site in ['site' + str(c) for c in range(1,11)]:
            if v[site] in site_dic and len(site_dic[v[site]]) == 1:
                data.set_value(r, -1, list(site_dic[v[site]])[0], takeable=True)
                continue
            elif v[site] in site_dic and len(site_dic[v[site]]) < 1:
                predictions = sorted([[int(user), float(user_dic[user][v[site]]) / sum(user_dic[user].values())] \
                                     for user in site_dic[v[site]] if v[site] in user_dic[user]], key=lambda t: t[1], reverse=True)
                if len(predictions):
                    session_prediction.append(predictions[0])
        if v["prediction"] == 0 and len(session_prediction):
            prediction = sorted(session_prediction, key=lambda t: t[1], reverse=True)[0][0]
            data.set_value(r, -1, prediction, takeable=True)

In [173]:
test_data[['site' + str(i) for i in range(1,11)]][40151:40154]

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
40151,2005,38,38,38,111,189,151,38,38,111
40152,32,32,32,32,32,0,0,0,0,0
40153,32,8,38,3837,82,0,0,0,0,0


In [174]:
train_data.shape

(91093, 40)

**Теперь у нас есть новый train и test, посчитанные с большим числом признаков. Сделаем мешок слов из сайтов и добавим остальные признаки.**

In [8]:
import pickle
pkl_file = open('kaggle_data/full_train.pkl', 'rb')
train_data = pickle.load(pkl_file)

pkl_file = open('kaggle_data/full_test.pkl', 'rb')
test_data = pickle.load(pkl_file)

In [200]:
#full train
train_test_data_sites = pd.concat([train_data[['site' + str(i) for i in range(1,11)] + ['site_longest_time']], \
                                   test_data[['site' + str(i) for i in range(1,11)] + ['site_longest_time']]])

In [6]:
#full train
train_test_data_sites = pd.concat([train_data[['site' + str(i) for i in range(1,11)]], \
                                   test_data[['site' + str(i) for i in range(1,11)]]])

In [180]:
#toy
train_test_data_sites = pd.concat([train_data_toy[['site' + str(i) for i in range(1,11)] + ['site_longest_time']], \
                                   test_data[['site' + str(i) for i in range(1,11)] + ['site_longest_time']]])

In [165]:
#toy
train_test_data_sites = pd.concat([train_data_toy[['site' + str(i) for i in range(1,11)]], \
                                   test_data[['site' + str(i) for i in range(1,11)]]])

In [177]:
print len(train_test_data_sites)
print(len(train_data), len(test_data))

64076
(91093, 41177)


In [9]:
#full train
X_train_test_sparse = sparsematrix(train_test_data_sites.as_matrix())
X_train_sparse = X_train_test_sparse[:len(train_data)]
X_test_sparse = X_train_test_sparse[len(train_data):]
y = train_data["target"]

In [166]:
#toy
X_train_test_sparse = sparsematrix(train_test_data_sites.as_matrix())
X_train_sparse = X_train_test_sparse[:len(train_data_toy)]
X_test_sparse = X_train_test_sparse[len(train_data_toy):]
y = train_data_toy["target"]

In [11]:
from scipy.sparse import coo_matrix, hstack
import scipy

In [131]:
train_features = coo_matrix(train_data[["session_timespan", "#unique_sites", "day_of_week", 'start_hour', "daytime", "fb_portion", "youtube_portion",\
           "top30_portion", "bot30_portion"]].values, dtype=float)

test_features = coo_matrix(test_data[["session_timespan", "#unique_sites", "day_of_week", 'start_hour', "daytime", "fb_portion", "youtube_portion",\
           "top30_portion", "bot30_portion"]].values, dtype=float)

In [12]:
train_features = coo_matrix(train_data[["#unique_sites", "day_of_week", 'start_hour', 'prediction']].values, dtype=int)

test_features = coo_matrix(test_data[["#unique_sites", "day_of_week", 'start_hour', 'prediction']].values, dtype=int)

In [181]:
#toy
train_features = coo_matrix(train_data_toy[["day_of_week", 'daytime', "prediction"]].values, dtype=int)

#test_features = coo_matrix(test_data_toy[["#unique_sites", 'site_longest_time', "day_of_week", 'start_hour']].values, dtype=int)

In [182]:
X_train_sparse.shape

(22899, 17610)

In [13]:
X_train_sparse = hstack([X_train_sparse, train_features])

In [184]:
X_train_sparse.shape

(22899, 17613)

In [14]:
X_test_sparse = hstack([X_test_sparse, test_features])

In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_sparse, y, test_size=0.3, 
                                                     random_state=7, stratify=y)

In [16]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.feature_selection import SelectFromModel

In [17]:
skf = StratifiedKFold(n_splits=5, shuffle=True)

In [69]:
X_train.shape

(16029, 17610)

**LinearSVC on GridSearchCV**

In [75]:
%%time
#"#unique_sites", "site_longest_time", "day_of_week", "start_hour"
# 1. Добавим только site_longest_time и проведём l1 регуляризацию для отбора признаков. 
# 2. Удалим признаки, отброшенные l1 регуляризатором, проведём l2. Результат CV: 0.485307879468, отложенная: 0.48922852983988357,
#                                                                                    'C': 0.1061578947368421, 'dual': False
# 3. Начнём сначала, добавим дополнительные признаки, снова проведём l1 Результат отложенной: 0.483697234352
# "#unique_sites", "day_of_week", 'start_hour', 'daytime'
# 4. Удалим нулевые признаки, применим l2. Результат CV: 0.486493231019, отложенная: 0.488937409025
#                                                                                  'C': 0.1061578947368421, 'dual': True
# 5. Повторим на всей выборке

svm = LinearSVC()
#svm_params2 = {'C': np.linspace(0.001, 1, 20), 'max_iter':[1000, 5000], 'dual': [True, False], 'loss': ['squared_hinge']}
#svm_params2 = {'C': np.linspace(1e-3, 1, 20), 'dual': [False], 'penalty': ['l1']}
svm_params2 = {'C': np.linspace(1e-3, 1, 20), 'dual': [True, False], 'penalty': ['l2']}


svm_grid_searcher2 = GridSearchCV(svm, param_grid=svm_params2, cv=skf, n_jobs=-1)
svm_grid_searcher2.fit(X_train_new, y_train)

print(svm_grid_searcher2.best_params_)
print(svm_grid_searcher2.best_score_)
print accuracy_score(y_valid, svm_grid_searcher2.predict(X_valid_new))

{'penalty': 'l2', 'C': 0.1061578947368421, 'dual': True}
0.486493231019
0.488937409025
CPU times: user 9.88 s, sys: 508 ms, total: 10.4 s
Wall time: 12min 26s


In [71]:
print accuracy_score(y_valid, svm_grid_searcher2.predict(X_valid))

0.483697234352


In [188]:
%%time
svm2 = LinearSVC(C=0.1061578947368421, dual=True, penalty='l2')
#svm2.fit(X_train_new, y_train)
svm_cv_scores = cross_val_score(svm, X_train, y_train, cv=skf, n_jobs=-1)
print svm_cv_scores, mean(svm_cv_scores)

[ 0.51074939  0.5122102   0.50826317  0.50833595  0.50634518] 0.50918077867
CPU times: user 684 ms, sys: 124 ms, total: 808 ms
Wall time: 22.2 s


In [210]:
svm2.fit(X_train_new, y_train)

LinearSVC(C=0.106157894737, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [211]:
accuracy_score(y_valid, svm2.predict(X_valid_new))

0.17732728337236534

In [ ]:
#svm.fit(X_train_new, y_train) LOOK HERE!
print accuracy_score(svm2.predict(X_valid_new), y_valid)

In [208]:
%%time
svm = LinearSVC(C=0.42163157894736841, dual=False, penalty='l1')
svm.fit(X_train, y_train)
print accuracy_score(svm.predict(X_valid), y_valid)

0.347445843091
CPU times: user 10min 38s, sys: 216 ms, total: 10min 38s
Wall time: 10min 38s


In [139]:
accuracy_score(svm.predict(X_valid), y_valid) # Score on full train with l1 regularization 0.35289812646370022

0.35432523419203749

In [111]:
X_train.shape

(63765, 24056)

In [209]:
model = SelectFromModel(svm, prefit=True)
X_train_new = model.transform(X_train)
print X_train_new.shape
X_valid_new = model.transform(X_valid)
print X_valid_new.shape
X_train_sparse_new = model.transform(X_train_sparse)
X_test_sparse_new = model.transform(X_test_sparse)

(63765, 12829)
(27328, 12829)


In [237]:
print(svm_grid_searcher2.best_params_)
print(svm_grid_searcher2.best_score_)
accuracy_score(y_valid, svm_grid_searcher2.predict(X_valid))
#"site_longest_time"

{'C': 0.1061578947368421}
0.324331529836


0.33116217798594849

In [231]:
print(svm_grid_searcher2.best_params_)
print(svm_grid_searcher2.best_score_)
accuracy_score(y_valid, svm_grid_searcher2.predict(X_valid))
#"#unique_sites", "site_longest_time", "day_of_week", "start_hour"

{'C': 0.1061578947368421}
0.344217046969


0.35260538641686184

In [193]:
print(svm_grid_searcher2.best_params_)
print(svm_grid_searcher2.best_score_)

{'C': 0.1061578947368421}
0.51044980972


**SGDClassifier**

In [256]:
%%time
sgd = SGDClassifier(n_jobs=-1)

#params1 = {'n_iter': [5], 'loss': ['hinge', 'log', 'squared_hinge'], \
               #'penalty': ['l2','l1','elasticnet'], 'l1_ratio': [0.01, 0.15, 0.5, 0.85]}

params1 = {'n_iter': [20], 'loss': ['squared_hinge'], \
               'penalty': ['elasticnet']}

svm_grid_searcher1 = GridSearchCV(sgd, param_grid=params1, cv=skf, n_jobs=-1)
svm_grid_searcher1.fit(X_train, y_train)

print(svm_grid_searcher1.best_params_)
print(svm_grid_searcher1.best_score_)

pred = svm_grid_searcher1.predict(X_valid)
print(accuracy_score(y_valid, pred))

{'penalty': 'elasticnet', 'loss': 'squared_hinge', 'n_iter': 20}
0.218599545205
0.230020491803
CPU times: user 3min 31s, sys: 292 ms, total: 3min 32s
Wall time: 2min 48s


- {'penalty': 'elasticnet', 'l1_ratio': 0.15, 'n_iter': 5, 'loss': 'log'}
- 'site_longest_time
- 0.440014972862
- 0.454148471616
- CPU times: user 4.55 s, sys: 304 ms, total: 4.86 s
- Wall time: 44.7 s

- {'n_iter': 5, 'eta0': 1, 'loss': 'squared_hinge', 'l1_ratio': 0.5, 'learning_rate': 'optimal', 'penalty': 'l1'}
- "#unique_sites", "day_of_week", 'start_hour'
- 0.409008671782
- 0.404075691412
- CPU times: user 32.4 s, sys: 1.24 s, total: 33.6 s
- Wall time: 16min 27s

**LogisticRegressionCV works too long on LARGE TRAINING SETS. Will try tuning it on 150 users. Still, multinomial logistic regression shows very poor result on full training set.**

In [90]:
%%time
logit_c_values1 = np.linspace(1e-3, 1, 10)
logit_c_values1 = [0.68996551724137933]

logit_grid_searcher1 = LogisticRegressionCV(Cs=logit_c_values1, multi_class='ovr', n_jobs=2, cv=skf)

logit_grid_searcher1.fit(X_train, y_train)

CPU times: user 10min 59s, sys: 3min 58s, total: 14min 57s
Wall time: 29min 11s


In [91]:
logit_mean_cv_scores1 = list(np.mean([i.mean(axis=0) for i in logit_grid_searcher1.scores_.values()], axis=0))
logit_mean_cv_scores1

[0.99836935390335713]

In [92]:
print("C:", logit_c_values1[logit_mean_cv_scores1.index(max(logit_mean_cv_scores1))], "score:", max(logit_mean_cv_scores1))

('C:', 0.6899655172413793, 'score:', 0.99836935390335713)


In [93]:
accuracy_score(y_valid, logit_grid_searcher1.predict(X_valid))

0.35075506445672189

In [94]:
write_to_submission_file(logit_grid_searcher1.predict(X_test_sparse), "kaggle_data/submission7.csv")
# Logistic Regression with C = 0.6899655172413793, 
# one-vs-rest, features="#unique_sites", "day_of_week", 'start_hour', score=0.38318

**Попробуем для начала LinearSVC с оптимальным параметром С, полученным на прошлой неделе, и без дополнительных признаков.**

In [19]:
from sklearn.svm import LinearSVC

In [20]:
%%time
svm = LinearSVC(C=0.10434482758620689, random_state=7)
svm.fit(X_train, y_train)

CPU times: user 53min 30s, sys: 116 ms, total: 53min 30s
Wall time: 53min 30s


In [23]:
%%time
svm_cv_scores = cross_val_score(svm, X_train, y_train, cv=skf, n_jobs=-1)

CPU times: user 11 s, sys: 1.4 s, total: 12.4 s
Wall time: 1h 26min 33s


In [24]:
svm_cv_scores #добавление prediction сильно ухудшило linearSVC c l2 и С= 0.10434482758620689

array([ 0.18919336,  0.18417166,  0.18932267,  0.17281707,  0.19304403])

In [16]:
%%time
accuracy_score(y_valid, svm.predict(X_valid))

CPU times: user 136 ms, sys: 24 ms, total: 160 ms
Wall time: 160 ms


0.3389318600368324

In [18]:
write_to_submission_file(svm.predict(X_test_sparse), "kaggle_data/submission2.csv")

In [12]:
a = [1, 4, 6, 3]
b = [9, 4, 1, 8]

c = zip(a, b)

In [13]:
c

[(1, 9), (4, 4), (6, 1), (3, 8)]

In [16]:
tuple([9, 1]) in c

False